In [1]:
from scipy import io
import os
import pandas as pd
import numpy as np
import random

import torch
from torch_geometric.data import Data, DataLoader

import math
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
import random
import numpy as np
from torch_geometric.nn import ChebConv, GCNConv
from torch_geometric.transforms import LaplacianLambdaMax
    

In [ ]:
# EEG_band : delta, theta, alpha, beta, gamma, all = 1,2,3,4,5,None
# Feature_name : de_LDS, PSD_LDS, etc.
def load_data(data_dir_path,label_dir_path, trial:int, feature_name, EEG_band=None):  
    subject_data_list, session_list = [], []
    file_list = os.listdir(data_dir_path)
    
    for idx, file in enumerate(file_list):
        trial_list = []
        data = io.loadmat(data_dir_path + file)
        if EEG_band is None:
            for trial_idx in range(1, trial+1):
                trial_list.append(data[feature_name + str(trial_idx)][:, :, :])  # EEG all bands

        else:
            for trial_idx in range(1, trial+1):
                trial_list.append(data[feature_name + str(trial_idx)][:, :, EEG_band])

        session_list.append(trial_list)

        if (idx + 1) % 3 == 0:
            subject_data_list.append(session_list)
            session_list = []

    label = io.loadmat(label_dir_path + 'label.mat')['label']
    label = [1 + label[0][i] for i in range(trial)]
    #label [-1, 1] --> [0, 2]

    # subject_data_list : num_subjects(15) × num_sessions(3) × num_trials(15)
    return subject_data_list, label


# edge attributes are composed of edge weights, the distance between all EEG channel pairs
def load_edge_information(pdc_dir_path, sub_name, pdc_var_name, n_channels, n_trials, n_sessions, n_subjects):

    file_list = os.listdir(pdc_dir_path)
    
    edge_index_list = []
    for i in range(n_channels):
        for j in range(n_channels):
            edge_index_list.append([i,j])
            
    edge_attr_list = []
    session_edge_attr_list = []
    sub_idx = 0
    for i, file in enumerate(file_list):
        trial_edge_attr_list = []
        pdcs = io.loadmat(pdc_dir_path + file)
        pdc_name = sub_name[sub_idx]+pdc_var_name
        for trial_idx in range(1, n_trials+1):
            edge_attr = []
            pdc = pdcs[pdc_name+str(trial_idx)][:,:]
            for k in range(n_channels):
                for l in range(n_channels):
                    if k == l:
                        edge_attr.append(0)
                    else:
                        edge_attr.append(pdc[k][l])
        
            trial_edge_attr_list.append(edge_attr)
        session_edge_attr_list.append(trial_edge_attr_list)
        if (i+1) % 3 == 0:
            sub_idx+=1
            edge_attr_list.append(session_edge_attr_list)
            session_edge_attr_list = []

    return edge_index_list, edge_attr_list


#Graph Representation
def get_graph_data(subject_data, subject_label, edge_index_list, edge_attr_list, num_train_trials , batch_size):
    edge_index = torch.tensor(edge_index_list, dtype=torch.long)
    train_loader, test_loader = [], []

    num_subjects = len(subject_data)
    num_sessions = len(subject_data[0])
    num_trials = len(subject_data[0][0])

    for subject in range(num_subjects):
        train_dataset, test_dataset = [], []
        for session in range(num_sessions):
            for trial in range(num_trials):
                data_list = []
                trial_data = subject_data[subject][session][trial] # trial_data = [62][about 240][1or5] = [nodes][trial time][EEG band(s)]
                blocks = len(trial_data[1]) # about 240(240 sec, 4minutes), 'blocks' is number of blocks which is the same as a trial time
                edge_attr = torch.tensor(edge_attr_list[subject][session][trial], dtype=torch.float)
                for block_idx in range(blocks):
                    # if using features of all frequency bands,
                    # node_feature = [delta, theta, alpha, beta, gamma]
                    data_sample = torch.tensor(trial_data[:, block_idx, :], dtype=torch.float)#data_sample = [62][5] = [nodes, node_features(EEG band(s)]
                    data_label = torch.tensor(subject_label[trial], dtype=torch.long) # data_label [0,2]
                    data_list.append(Data(x=data_sample, edge_index=edge_index.t().contiguous(), edge_attr=edge_attr, y=data_label))
                if trial < num_train_trials: 
                    train_dataset.extend(data_list)
                else: 
                    test_dataset.extend(data_list)
            
        random.shuffle(train_dataset)
        random.shuffle(test_dataset)
        
        batch_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        batch_test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        train_loader.append(batch_train_loader)
        test_loader.append(batch_test_loader)
        print('loading' + str(subject))
    print("\nTrain dataset length: {}, \tTest dataset legnth: {}".format(len(train_dataset), len(test_dataset)))
    return train_loader, test_loader


In [ ]:
def criterion(loss, label, model, L1_regularization_scaler):
    l1_regularization = torch.tensor(0., device=device)
    
    for param in model.parameters():
        l1_regularization += torch.norm(param, 1)

    loss += L1_regularization_scaler * l1_regularization
    
    return loss

def train(loader, model, optimizer, L1_regularization_scaler, epoch, batch_size):
    model.train()
    train_acc, train_loss, count = 0., 0., 0.

    dataset_length = len(loader.dataset)
    loader_length = len(loader)
    for batch_idx, data in enumerate(loader):
        if len(data.y) == batch_size: # dataset size가 batch size로 안나눠지면 버림
            count += 1.
            data = data.to(device)
            loss,result = model(data, 'train')
            loss = criterion(loss, data.y, model, L1_regularization_scaler)
            loss.backward()

#             with torch.no_grad():
#                 for p in model.parameters():
#                     #p = (1-lr)*p + lr*p.grad
#                     print(p.requires_grad," row : ", p)
#                     p = p.grad
#                     print(p.requires_grad, " copy : ",p)
#                     break
#             for p in model.parameters():
#                 print(p.requires_grad, " update: ", p)
#                 break
            optimizer.step()
            optimizer.zero_grad()
            train_loss += loss.item()
            pred = result.max(1, keepdim = True)[1]
            acc = pred.eq(data.y.view_as(pred)).sum().item()
            acc = 100. * acc / batch_size

            train_acc += acc

    train_acc /= count
    train_loss /= count
    return train_acc, train_loss


def test(loader, model, L1_regularization_scaler, batch_size):
    model.eval()
    count, test_loss, test_acc = 0, 0, 0
    with torch.no_grad():
        for data in loader:
            if len(data.y) == batch_size:
                count += 1
                data = data.to(device)
                output, result = model(data)
                loss = output.item()
                test_loss += loss
                
                pred = result.max(1, keepdim = True)[1]
                acc = pred.eq(data.y.view_as(pred)).sum().item()
                acc = 100. * acc / batch_size
                test_acc += acc

    test_acc /= count
    test_loss /= count
    return test_acc, test_loss

In [ ]:
class SSLGCN(nn.Module):
    def __init__(self, num_nodes,num_features, gcn1_channels, gcn2_channels, gcn3_channels, fc1_channels, out_channels, edge_weight, batch_size, learnable=False):
        super(SSLGCN, self).__init__()
        self.in_channels = num_features
        self.gcn1_out_channels = gcn1_channels
        self.gcn2_out_channels = gcn2_channels
        self.gcn3_out_channels = gcn3_channels
#         self.fc1_in_channels = (gcn1_channels + gcn2_channels) * num_nodes
        self.fc1_in_channels = gcn3_channels * num_nodes
        self.fc1_out_channels = fc1_channels
        self.out_channels = out_channels
        
        self.lambdamax = LaplacianLambdaMax(None)
        self.edge_weight = nn.Parameter(edge_weight, requires_grad=learnable)
        self.batch_size = batch_size
        self.softmax = nn.Softmax(dim=1)
        self.crossentropy = nn.CrossEntropyLoss()
        self.relu = nn.ReLU()
        self.leakyrelu = nn.LeakyReLU(0.15)
        self.dropout = nn.Dropout(0.1)
        
        self.bn1d = nn.BatchNorm1d(self.in_channels)
        self.gcn1 = GCNConv(self.in_channels, self.gcn1_out_channels)
        self.gcn2 = GCNConv(self.gcn1_out_channels, self.gcn2_out_channels)
        self.gcn3 = GCNConv(self.gcn2_out_channels, self.gcn3_out_channels)
        self.fc1 = nn.Linear(self.fc1_in_channels, self.fc1_out_channels)
        self.fc2 = nn.Linear(self.fc1_out_channels, self.out_channels)
        self.fc_module = nn.Sequential(self.fc1, self.dropout, self.leakyrelu, self.fc2)
        
    def forward(self, data, _type=None):
        data.edge_attr = self.edge_weight.data.repeat(self.batch_size)
        data = self.lambdamax(data)

        if data.x.dim() == 1:
            data.x = data.x.unsqueeze(dim=1)
        
        bn = self.leakyrelu(self.bn1d(data.x))
        gcn1 = self.gcn1(bn, data.edge_index, self.leakyrelu(data.edge_attr))
        gcn2 = self.gcn2(self.leakyrelu(gcn1), data.edge_index, self.leakyrelu(data.edge_attr))
        gcn3 = self.gcn3(self.leakyrelu(gcn2), data.edge_index, self.leakyrelu(data.edge_attr))
        
#         gcn = torch.cat((gcn1, gcn2), 0)
        gcn = self.leakyrelu(gcn3).reshape(self.batch_size, -1)

        fc = self.fc_module(gcn)

        loss = self.crossentropy(fc.view(-1, self.out_channels), data.y.view(-1))
        result = self.softmax(fc)
        
        return loss, result

class DGCNN(nn.Module): # Two GCN with Batch Normalization and LeakyReLU
    def __init__(self, num_nodes,num_features, hid_channels, out_channels, k,  edge_weight, batch_size, learnable=False):
        super(DGCNN, self).__init__()

        self.lambdamax = LaplacianLambdaMax(None)

        self.in_channels = num_features
        self.cheb_out_channels = hid_channels

        self.fc1_in_channels = 1240
        self.fc1_out_channels = 128 # 
        self.out_channels = out_channels

        self.edge_weight = nn.Parameter(edge_weight, requires_grad=learnable)
        self.batch_size = batch_size
        self.num_nodes = num_nodes
        
        self.chebconv1 = ChebConv(self.in_channels, self.cheb_out_channels, K=k, normalization = None)
        # batch normalization
        
        self.BN1d1 = nn.BatchNorm1d(self.in_channels)
        #self.BN1d1 = nn.BatchNorm1d(self.cheb_out_channels)
        
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.leakyrelu = nn.LeakyReLU(0.15)
        
        self.fc1 = nn.Linear(self.fc1_in_channels, self.fc1_out_channels)
        self.fc2 = nn.Linear(self.fc1_out_channels, self.out_channels)
        self.fc_module = nn.Sequential(self.fc1, self.leakyrelu, self.fc2)

        
                                      
    def forward(self, data, _type=None):
        data.edge_attr = self.edge_weight.data.repeat(self.batch_size)
        data = self.lambdamax(data)

        if data.x.dim() == 1:
            data.x = data.x.unsqueeze(dim=1)

        data.x = self.leakyrelu(self.BN1d1(data.x))

        cheb_layer = self.chebconv1(data.x, data.edge_index, self.leakyrelu(data.edge_attr), lambda_max=data.lambda_max)

        cheb_layer = self.leakyrelu(cheb_layer).reshape(self.batch_size, -1)
        fc_layer = self.fc_module(cheb_layer)
        
        if _type == 'train':
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(fc_layer.view(-1, self.out_channels), data.y.view(-1))
            logits = self.softmax(fc_layer)
            return loss, logits
        else:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(fc_layer.view(-1, self.out_channels), data.y.view(-1))
            logits = self.softmax(fc_layer)
            return loss, logits
    

In [ ]:
data_dir_path = 'D:/fragmentation project/Emotion_Recognition/DGCNN/GCN_implement/GCN_implement/dataset/SEED/SEED/ExtractedFeatures/data/'
feature_name = 'de_LDS'
# data_dir_path = 'C:/Users/daehyeon/Desktop/GCN_implement/GCN_implement/dataset/SEED/SEED/ExtractedFeatures/data/'
# feature_name = 'de_LDS'
label_dir_path = 'D:/fragmentation project/Emotion_Recognition/DGCNN/GCN_implement/GCN_implement/dataset/SEED/SEED/ExtractedFeatures/label/'
channels_dir_path = 'D:/fragmentation project/Emotion_Recognition/DGCNN/GCN_implement/GCN_implement/requirements'

pdc_dir_path = 'E:/asympPDC-main/pdcs_nodiag/'
sub_name = ['djc', 'jl', 'jj', 'lqj', 'ly', 'mhw', 'phl', 'sxy', 'wk', 'ww', 'wsf', 'wyw', 'xyl', 'ys', 'zjy'];
pdc_var_name = '_PDC_mean'



In [ ]:
sessions, subjects, trials, num_nodes, num_classes, train_trials = 3, 15, 15, 62, 3, 9  # 논문과 데이터셋에서 명시된 값
epochs, freq_bands, random_seed = 100, 5, 42  # training 할 때 변경이 가능한 hyper-parameter
k, batch_size, learning_rate, lambda1 = 3, 32, 0.01, 0.001  # training 할 때 변경이 가능한 parameter
EEG_band = None
subject_data, subject_label = load_data(data_dir_path,label_dir_path, trials, feature_name, EEG_band)
edge_index_list, edge_attr_list = load_edge_information(pdc_dir_path, sub_name, pdc_var_name, num_nodes, trials, sessions, subjects)
train_loader, test_loader = get_graph_data(subject_data, subject_label,
                                 edge_index_list, edge_attr_list,
                                 train_trials , batch_size)


In [ ]:
sub_train_loader_pdc = train_loader[4]
sub_edge_attr_pdc = sub_train_loader_pdc.dataset[4].edge_attr
sub_test_loader_pdc = test_loader[4]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
epochs = 200

# prev = 0
# best_acc = np.zeros((15), float)

for sub in range (10,11):
    cnt = 0
    learning_rate = 0.0001

    sub_train_loader_pdc = train_loader[sub]
    sub_edge_attr_pdc = sub_train_loader_pdc.dataset[sub].edge_attr
    sub_test_loader_pdc = test_loader[sub]
    len_lr = 3;
    len_ld = 5;
    
    f = open('E:/pdc_result/all_sub_dgcnn/continuesss_22-08-31 sub' + str(sub+1) + '_DGCNN_PDC_epoch200.txt', 'w')
    
    for lr in range (len_lr):
        lambda1 = 0.001
        
        for ld in range(len_ld):
    #         learning_check = 0
            cnt+=1
            print("OK {}".format(cnt))
            w = "lr: "+str(learning_rate) + "  ld: "+str(lambda1) + "\n--------------------------------------------------------------------\n"
            f.write(w)
            model6 = DGCNN6(num_nodes, freq_bands, 20, num_classes, k, sub_edge_attr_pdc, batch_size, True).to(device)
            optimizer6 = torch.optim.Adam(model6.parameters(), lr=learning_rate)#, weight_decay=5e-3)
            for epoch in range(1, epochs+1):
                train_acc, train_loss = train(sub_train_loader_pdc, model6, optimizer6, lambda1, epoch, batch_size)

                if epoch % 10 == 0:
                    a = "Epoch: "+str(epoch)+", Train Acc: " + str(round(train_acc,2)) + ",Train Loss: " + str(round(train_loss,4)) + "\n"
                    f.write(a)
                test_acc, test_loss = test(sub_test_loader_pdc, model6, lambda1, batch_size)
                if test_acc > best_acc[sub]:
                    best_acc[sub] = test_acc
                b = "Epoch: "+str(epoch)+", Model6 Evaluation Result -  Test Accuracy: " + str(round(test_acc,2)) + ",\tTest Loss: " + str(round(test_loss,4)) + "\n\n"
                f.write(b)

    #             if str(test_acc)[:5] == str(prev)[:5]:
    #                 learning_check += 1
    #             else:
    #                 learning_check = 0
    #             if learning_check == 8:
    #                 break

    #             prev = test_acc
            lambda1 /= 10
            f.write("------------------------------------------------------------------\n\n")
        f.write("------------------------------------------------------------------\n\n\n")
        learning_rate /= 10
    f.write("!!! BEST ACCURACY !!!\n")
    f.write(str(best_acc[sub]))
    f.write("\n")
    f.close()
    
print("\n-------- Best Acc ---------------")
print(best_acc)

f = open('E:/pdc_result/all_sub_dgcnn/22-08-31 sub_acc_mean_std', 'w')
mean_acc = best_acc.mean()
std_acc = best_acc.std()
f.write("mean_acc : ")
f.write(str(mean_acc))
f.write("std_acc: ")
f.write(str(std_acc))
f.close()

print("\n-------------------------------")
print("mean_acc : ", mean_acc)
print("std_acc: ", std_acc)